# Linear Model 3 - Linear SVM

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from auxiliars import *
import pickle

## Data

In [2]:
data = pd.read_csv("./data/stdHTRU_2.csv")

In [3]:
col = data['class'].map({1:'r', 0:'b'})
pd.plotting.scatter_matrix(data.drop(['class'], axis = 1), c=col, figsize=(15,15))

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x11b062b70>,
      dtype=object)

From the Scatter Matrix we can de deduce that the Linear Kernel should be sufficient for the separation of classes.

Even so, we can obvserve that some features, see for example DM_mean-DM_stdev, have very close data. In order to reduce the impact of this fact, let's train SVM with (standarized) normal data and data with selected features. 

We split a separate test set of relative size 20%:

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data[data.columns[0:8]], 
                                                    data['class'], 
                                                    test_size = 0.2,
                                                    random_state = 1234)

We will analyze the performance of the method with no-correlated standarized data: 

In [5]:
noCorrData = pd.read_csv("./data/noCorrStdHTRU_2.csv")

In [6]:
X_train_NC, X_test_NC, y_train_NC, y_test_NC = train_test_split(noCorrData[noCorrData.columns[0:8]], 
                                                    noCorrData['class'], 
                                                    test_size = 0.2,
                                                    random_state = 1234)

## Model Training

In order to train Linear SVM we are going to use the scikit-learn LinearSVC class, specialized in Linear SVM.

In [7]:
from sklearn.svm import LinearSVC

In [8]:
SVMClass = LinearSVC(random_state = 1234, max_iter = 5000)

LinearSVC allow us to hypertuning the following parameters:
- Regularization parameter C.
- Class weights:
    - Dict: Weights specified by class.
    - Balanced: Uses the values of target (y) to automatically adjust weights inversely proportional to class frequencies in the input data.

In order to hypertuning model parameters and get a better idea on how the model performs on unseen data, we will use GridSearchCV.

In [9]:
from sklearn.model_selection import GridSearchCV

Values of the 10-Fold CV Grid to test:

In [10]:
grid = {'C': [10**x for x in range(-3, 4, 1)],
        'class_weight': [{0: 1, 1: 1}, 'balanced']}

In [11]:
grid

{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
 'class_weight': [{0: 1, 1: 1}, 'balanced']}

Grid Search 10-Fold CV:

In [12]:
gs10cv = GridSearchCV(SVMClass, param_grid = grid, cv = 10, n_jobs = -1)

### Normal Data Training

In [13]:
gs10cv.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=5000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=1234, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'class_weight': [{0: 1, 1: 1}, 'balanced']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [14]:
pd.DataFrame(gs10cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.094518,0.033987,0.009792,0.011075,0.001,"{0: 1, 1: 1}","{'C': 0.001, 'class_weight': {0: 1, 1: 1}}",0.972067,0.977654,0.973464,0.972067,0.968575,0.977654,0.977654,0.969972,0.976939,0.973445,0.973949,0.003200,6
1,0.099221,0.018246,0.005319,0.005349,0.001,balanced,"{'C': 0.001, 'class_weight': 'balanced'}",0.969274,0.974162,0.967877,0.969274,0.970670,0.973464,0.968575,0.966480,0.966457,0.966457,0.969269,0.002636,14
2,0.121129,0.026519,0.003550,0.000964,0.01,"{0: 1, 1: 1}","{'C': 0.01, 'class_weight': {0: 1, 1: 1}}",0.974162,0.979050,0.974162,0.976257,0.970670,0.979050,0.980447,0.973464,0.980433,0.975542,0.976324,0.003144,5
3,0.337977,0.072747,0.008785,0.006032,0.01,balanced,"{'C': 0.01, 'class_weight': 'balanced'}",0.974162,0.976955,0.974860,0.970670,0.972765,0.976257,0.972067,0.972765,0.972048,0.967855,0.973040,0.002546,10
4,0.491656,0.069638,0.005742,0.005282,0.1,"{0: 1, 1: 1}","{'C': 0.1, 'class_weight': {0: 1, 1: 1}}",0.976257,0.980447,0.975559,0.978352,0.975559,0.981844,0.980447,0.975559,0.979734,0.979036,0.978279,0.002260,3
5,3.275689,0.672979,0.003497,0.001143,0.1,balanced,"{'C': 0.1, 'class_weight': 'balanced'}",0.974860,0.976257,0.974162,0.972765,0.974162,0.973464,0.970670,0.972765,0.971349,0.970650,0.973110,0.001751,9
6,2.781144,0.428611,0.003461,0.000494,1,"{0: 1, 1: 1}","{'C': 1, 'class_weight': {0: 1, 1: 1}}",0.976955,0.981145,0.975559,0.979050,0.976257,0.982542,0.981145,0.976257,0.979734,0.979734,0.978838,0.002316,1
7,15.512217,1.303989,0.003699,0.000927,1,balanced,"{'C': 1, 'class_weight': 'balanced'}",0.974162,0.974860,0.972067,0.972067,0.974162,0.973464,0.970670,0.969972,0.969951,0.969252,0.972063,0.001931,11
8,8.078849,1.802387,0.009260,0.011697,10,"{0: 1, 1: 1}","{'C': 10, 'class_weight': {0: 1, 1: 1}}",0.976955,0.981145,0.975559,0.979050,0.976257,0.982542,0.981145,0.976955,0.979036,0.979734,0.978838,0.002231,2
9,17.212021,2.728350,0.003417,0.001506,10,balanced,"{'C': 10, 'class_weight': 'balanced'}",0.972765,0.974860,0.969972,0.972067,0.974162,0.973464,0.970670,0.969972,0.969951,0.969252,0.971714,0.001911,12


In [15]:
gs10cv.best_params_

{'C': 1, 'class_weight': {0: 1, 1: 1}}

In [16]:
pd.DataFrame(gs10cv.cv_results_).iloc[gs10cv.best_index_]

mean_fit_time                                        2.78114
std_fit_time                                        0.428611
mean_score_time                                   0.00346105
std_score_time                                   0.000494467
param_C                                                    1
param_class_weight                              {0: 1, 1: 1}
params                {'C': 1, 'class_weight': {0: 1, 1: 1}}
split0_test_score                                   0.976955
split1_test_score                                   0.981145
split2_test_score                                   0.975559
split3_test_score                                    0.97905
split4_test_score                                   0.976257
split5_test_score                                   0.982542
split6_test_score                                   0.981145
split7_test_score                                   0.976257
split8_test_score                                   0.979734
split9_test_score       

In [17]:
# Save model
SVMClassFile = open('./models/SVMClass_BestCV_STDData_pickle_file', 'wb')
pickle.dump(gs10cv, SVMClassFile) 

### No-correlated Data Training

Grid Search 10-Fold CV:

In [18]:
gs10cv_nc = GridSearchCV(SVMClass, param_grid = grid, cv = 10, n_jobs = -1)

Training:

In [19]:
gs10cv_nc.fit(X_train_NC, y_train_NC)

GridSearchCV(cv=10, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=5000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=1234, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'class_weight': [{0: 1, 1: 1}, 'balanced']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [20]:
pd.DataFrame(gs10cv_nc.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.056879,0.009894,0.003963,0.002406,0.001,"{0: 1, 1: 1}","{'C': 0.001, 'class_weight': {0: 1, 1: 1}}",0.993715,0.995810,0.990223,0.990922,0.990223,0.990922,0.993715,0.992318,0.993012,0.992313,0.992317,0.001710,14
1,0.062925,0.004568,0.004526,0.003005,0.001,balanced,"{'C': 0.001, 'class_weight': 'balanced'}",0.999302,0.998603,0.998603,0.998603,0.998603,1.000000,1.000000,0.999302,0.998602,0.997904,0.998952,0.000644,13
2,0.063724,0.006159,0.002992,0.000355,0.01,"{0: 1, 1: 1}","{'C': 0.01, 'class_weight': {0: 1, 1: 1}}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
3,0.227859,0.034579,0.003272,0.000557,0.01,balanced,"{'C': 0.01, 'class_weight': 'balanced'}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
4,0.255873,0.027986,0.003559,0.001547,0.1,"{0: 1, 1: 1}","{'C': 0.1, 'class_weight': {0: 1, 1: 1}}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
5,1.490832,0.221211,0.005019,0.004049,0.1,balanced,"{'C': 0.1, 'class_weight': 'balanced'}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
6,2.308800,0.506626,0.006910,0.007371,1,"{0: 1, 1: 1}","{'C': 1, 'class_weight': {0: 1, 1: 1}}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
7,6.322122,1.393917,0.003107,0.000310,1,balanced,"{'C': 1, 'class_weight': 'balanced'}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
8,10.384394,2.040363,0.003056,0.000341,10,"{0: 1, 1: 1}","{'C': 10, 'class_weight': {0: 1, 1: 1}}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
9,14.609827,2.906608,0.002949,0.000078,10,balanced,"{'C': 10, 'class_weight': 'balanced'}",1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1


In [21]:
gs10cv_nc.best_params_

{'C': 0.01, 'class_weight': {0: 1, 1: 1}}

In [22]:
pd.DataFrame(gs10cv_nc.cv_results_).iloc[gs10cv_nc.best_index_]

mean_fit_time                                         0.0637244
std_fit_time                                         0.00615944
mean_score_time                                      0.00299177
std_score_time                                      0.000354517
param_C                                                    0.01
param_class_weight                                 {0: 1, 1: 1}
params                {'C': 0.01, 'class_weight': {0: 1, 1: 1}}
split0_test_score                                             1
split1_test_score                                             1
split2_test_score                                             1
split3_test_score                                             1
split4_test_score                                             1
split5_test_score                                             1
split6_test_score                                             1
split7_test_score                                             1
split8_test_score                       

In [23]:
# Save model
SVMClassFileNC = open('./models/SVMClass_BestCV_NCorrSTDData_pickle_file', 'wb')
pickle.dump(gs10cv_nc, SVMClassFileNC) 

## Testing

### Normal Data Model Testing

In [24]:
y_pred = gs10cv.predict(X_test)

In [25]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      3249
           1       0.93      0.80      0.86       331

    accuracy                           0.98      3580
   macro avg       0.96      0.90      0.92      3580
weighted avg       0.98      0.98      0.98      3580



In [26]:
print ("Confusion Matrix:")
confusionMatrix(y_test, y_pred, classes = [0,1])

Confusion Matrix:


Predicted,0,1
Real,,
0,3229,20
1,65,266


In [27]:
print("Test Error:")
(1-accuracy_score(y_test, gs10cv.predict(X_test)))*100

Test Error:


2.3743016759776525

### No-correlated Data Model Testing

In [28]:
y_pred_NC = gs10cv_nc.predict(X_test_NC)

In [29]:
print(classification_report(y_test_NC, y_pred_NC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3249
           1       1.00      1.00      1.00       331

    accuracy                           1.00      3580
   macro avg       1.00      1.00      1.00      3580
weighted avg       1.00      1.00      1.00      3580



In [30]:
print ("Confusion Matrix:")
confusionMatrix(y_test_NC, y_pred_NC, classes = [0,1])

Confusion Matrix:


Predicted,0,1
Real,,
0,3249,0
1,0,331


In [31]:
print("Test Error:")
(1-accuracy_score(y_test_NC, gs10cv_nc.predict(X_test_NC)))*100

Test Error:


0.0